In [3]:
from google.colab import files
from IPython.display import clear_output
!pip install pycodestyle pycodestyle_magic
!pip install flake8
clear_output()
%load_ext pycodestyle_magic
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [4]:
import re
import nltk
import joblib

import pandas as pd
import numpy as np

from pprint import pprint
from ast import literal_eval
from datetime import datetime

from tqdm.notebook import tqdm

from nltk.corpus import stopwords

from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

In [5]:
whole_table_with_lemm = drive.CreateFile(
    {'id': '1RD1lvRV3oHrWfI9igZBqQYlY0fLWjpa7'})
whole_table_with_lemm.GetContentFile('whole_table_with_lemm.csv')

In [6]:
pd.options.display.max_colwidth = 100

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
stop_words = set(stopwords.words('russian'))
new_stop_words = {
    'ваш', 'весь', 'который', 'наш', 'очень', 'самый', 'свой', 'твой', 'это'
    }

stop_words |= new_stop_words
len(stop_words)

160

In [9]:
def date_parser(date):
    if date and re.match(r'^(19|20)\d{2}/[1-9]\d?/[1-9]\d?$', date):
        date_split = [int(elem) for elem in date.split('/')]
        if date_split[0] >= 1939 and date_split[0] <= 1953:
            return datetime(year=date_split[0],
                            month=date_split[1],
                            day=date_split[2])
    return np.nan

In [10]:
df = pd.read_csv('whole_table_with_lemm.csv', sep='\t', index_col=0,
                 usecols=[0, 2, 3], parse_dates=['dates'],
                 date_parser=date_parser).rename(
                     columns={'notes\n': 'lemm', 'dates': 'date'})

In [11]:
df.head()

,lemm,date
0,"воскресение. \nвчера приезжать слава навещать мать в бол-це, но слезать с поезд в толмачев, г...",NaT
1,"ночь мы достигать банка антон-дорн, в датский пролив (между исландия и гренландия). утро погода ...",NaT
2,суббота. москва. ехать в селище.\n,NaT
3,"18 август его королевский величество отправляться из вильна в орша. в тот же день, высылать пере...",NaT
4,"на следующий день, 19 август, прибывать в остров, где не быть никакой известие.\n",NaT


In [12]:
df = df.dropna().set_index('date')

In [13]:
df.head()

,lemm
date,
1939-01-01,"вчера: елка зажигать. сергей ликовать. борис робертович приносить французский шампанское, на зво..."
1939-01-01,"январь 39 г. хотеть свой опыт по объект «вредительство» продолжать далее, быть еще ванна «гоган»..."
1939-01-01,новогодний состояние. пошлый мысль. \nбыть б. ш. как всегда с интерес болтать. \nчитать ил...
1939-01-01,"кроме один день (2-го, когда быть у парикмахер и зубной врач), сидеть дома и писать «восточный в..."
1939-01-01,"делать журнал для экспорт. ходить и возмущаться, весь начальник говорить неприятность. а не гово..."


In [14]:
regex = r'[!\"#$%&\'()*+,\-./:;<=>?@[\\\]^_`{|}~— \n\d\xa0«»–❉№°]+'

In [15]:
non_words = {
    'аа', 'ак', 'ап', 'аран', 'бол', 'го', 'игада', 'олхо', 'оп', 'отд', 'ошай', 'пог'
    }

In [17]:
def tokenization(text):
    return [elem for elem in re.split(regex, text.lower())
            if len(elem) > 1 
            and elem not in stop_words 
            and elem not in non_words 
            and elem not in ['id', 'com']]

In [18]:
df['lemm'] = df['lemm'].apply(tokenization)

In [20]:
df.head()

,lemm
date,
1939-01-01,"[вчера, елка, зажигать, сергей, ликовать, борис, робертович, приносить, французский, шампанское,..."
1939-01-01,"[январь, хотеть, опыт, объект, вредительство, продолжать, далее, ванна, гоган, калка, пружина, х..."
1939-01-01,"[новогодний, состояние, пошлый, мысль, интерес, болтать, читать, ильф, петров, полибия, удачный,..."
1939-01-01,"[кроме, день, парикмахер, зубной, врач, сидеть, дома, писать, восточный, война, страдать, грипп,..."
1939-01-01,"[делать, журнал, экспорт, ходить, возмущаться, начальник, говорить, неприятность, говорить, рабо..."


In [21]:
df2 = df.copy()

In [24]:
list_of_words = 'война'
key_words = list_of_words.split()
for war in key_words:
    df2[war] = df2.lemm.apply(lambda x: war in tuple(x))
df2 = df2[df2['война']]
del df2['война'] 
df2

,lemm
date,
1939-01-01,"[кроме, день, парикмахер, зубной, врач, сидеть, дома, писать, восточный, война, страдать, грипп,..."
1939-01-02,"[час, состояться, встреча, боец, участник, бой, озеро, хасан, встреча, показывать, кино, завтра,..."
1939-01-09,"[утро, автомобиль, академия, наука, узкий, вместе, нечкина, опушать, снег, парк, предупредительн..."
1939-01-14,"[русский, язык, вызывать, доска, ответ, получать, физика, получать, отлично, видеть, василий, со..."
1939-01-15,"[купить, кайзер, кончать, читать, книга, давать, летун, купить, кайзер, автор, некто, теодор, пл..."
...,...
1953-11-26,"[дата, автобиография, савченко, василий, алексеевич, рождаться, август, семья, крестьянин, село,..."
1953-11-29,"[день, подряд, ездить, анна, петровна, утро, писать, диктовка, впечатление, хотеться, успевать, ..."
1953-12-01,"[книга, прочитывать, начинать, лето, сентябрь, последующий, перечень, книга, объединять, фигурны..."


In [25]:
df2.to_csv('prozhito_lemmas.csv')

In [26]:
!zip prozhito_lemmas.zip prozhito_lemmas.csv

  adding: prozhito_lemmas.csv (deflated 79%)


In [27]:
df2 = [
    df2.loc[
        pd.Timestamp('1939-1-1 00:00:00'): pd.Timestamp('1941-6-21 23:59:59')
        ],
    df2.loc[
        pd.Timestamp('1941-6-22 00:00:00'): pd.Timestamp('1945-5-9 23:59:59')
        ],
    df2.loc[
        pd.Timestamp('1945-5-10 00:00:00'): pd.Timestamp('1953-12-31 23:59:59')
        ]]

In [28]:
df2[1].head()

,lemm
date,
1941-06-22,"[час, выступление, молотов, радио, война, германия, против, давно, ждать, уверенный, предсказыва..."
1941-06-22,"[идти, репетиция, макаренко, разрешение, врываться, анор, тревожный, глухой, голос, сообщать, во..."
1941-06-22,"[час, минута, страна, слышать, выступление, тов, молотов, сообщать, сегодня, час, утро, германск..."
1941-06-22,"[ночь, присниться, сон, начинаться, война, идти, поле, окоп, окоп, вылезать, немец, идти, просып..."
1941-06-22,"[война, начинаться, война, фашистский, германия, немец, переходить, граница, идти, бой, четыре, ..."


In [29]:
period = ['before Great Patriotic War', 'Great Patriotic War', 'After Patriotic War']

In [30]:
for i, time in enumerate(period):
    print(f'{time}:{" " * (20 - len(time))}\t{df2[i].shape[0]} entries')

before Great Patriotic War:	970 entries
Great Patriotic War: 	8063 entries
After Patriotic War: 	1312 entries


In [34]:
for i, table in tqdm(enumerate(df2)):
    table.to_csv(f'{i + 1}_{period[i].replace(" ", "_")}.csv')

0it [00:00, ?it/s]

In [35]:
!rm whole_table_with_lemm.csv
!rm prozhito_lemmas.csv
!zip prozhito_periodization.zip *.csv

rm: cannot remove 'whole_table_with_lemm.csv': No such file or directory
rm: cannot remove 'prozhito_lemmas.csv': No such file or directory
  adding: 1_before_Great_Patriotic_War.csv (deflated 79%)
  adding: 2_Great_Patriotic_War.csv (deflated 79%)
  adding: 3_After_Patriotic_War.csv (deflated 79%)


In [36]:
# код для загрузки архива таблиц для каждого периода

prozhito_periodization = drive.CreateFile(
   {'id': '1ghZXSjw92FIzpAIewSDZGkfJf8IpJ7u9'})
prozhito_periodization.GetContentFile('prozhito_periodization.zip')

In [40]:
models = []
for table in df2:
    data = list(table['lemm'])
    id2word = Dictionary(data)
    corpus = [id2word.doc2bow(text) for text in data]
    model = LdaModel(corpus, num_topics=10, id2word=id2word, passes=15,
                     iterations=100, per_word_topics=True)
    models.append(model)
    clear_output()
    print(f'Моделей обучено: {len(models)}')

Моделей обучено: 3


In [41]:
joblib.dump(models, 'prozhito_topic_modelling.pkl')

['prozhito_topic_modelling.pkl']

In [42]:
!zip prozhito_topic_modelling.zip prozhito_topic_modelling.pkl

  adding: prozhito_topic_modelling.pkl (deflated 44%)


In [43]:
files.download('prozhito_topic_modelling.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:

pprint(models[0].print_topics())

[(0,
  '0.011*"война" + 0.007*"любовь" + 0.007*"мочь" + 0.006*"человек" + '
  '0.006*"ляля" + 0.006*"сказать" + 0.005*"жизнь" + 0.004*"говорить" + '
  '0.004*"год" + 0.004*"знать"'),
 (1,
  '0.010*"война" + 0.007*"германия" + 0.006*"мочь" + 0.006*"гитлер" + '
  '0.005*"англия" + 0.005*"франция" + 0.004*"год" + 0.004*"время" + '
  '0.004*"день" + 0.004*"сегодня"'),
 (2,
  '0.005*"день" + 0.005*"москва" + 0.005*"говорить" + 0.004*"год" + '
  '0.004*"война" + 0.004*"мочь" + 0.004*"de" + 0.004*"человек" + 0.004*"финн" '
  '+ 0.003*"гюнтер"'),
 (3,
  '0.006*"мочь" + 0.006*"говорить" + 0.005*"самолет" + 0.005*"немецкий" + '
  '0.005*"знать" + 0.005*"несколько" + 0.004*"сказать" + 0.004*"день" + '
  '0.004*"видеть" + 0.004*"время"'),
 (4,
  '0.006*"человек" + 0.005*"сказать" + 0.004*"война" + 0.004*"мочь" + '
  '0.004*"говорить" + 0.003*"год" + 0.003*"покушение" + 0.003*"день" + '
  '0.003*"время" + 0.003*"слово"'),
 (5,
  '0.009*"немец" + 0.007*"война" + 0.006*"армия" + 0.005*"германский" + 

In [45]:

pprint(models[1].print_topics())

[(0,
  '0.025*"день" + 0.022*"война" + 0.021*"победа" + 0.010*"год" + 0.009*"ночь" '
  '+ 0.007*"ура" + 0.007*"друг" + 0.006*"кончаться" + 0.006*"город" + '
  '0.006*"огромный"'),
 (1,
  '0.013*"человек" + 0.009*"война" + 0.008*"жизнь" + 0.008*"мир" + '
  '0.007*"мочь" + 0.007*"народ" + 0.006*"день" + 0.006*"русский" + 0.006*"год" '
  '+ 0.005*"германия"'),
 (2,
  '0.012*"война" + 0.008*"фронт" + 0.007*"год" + 0.007*"армия" + '
  '0.005*"сталин" + 0.005*"советский" + 0.005*"германия" + 0.005*"военный" + '
  '0.004*"мочь" + 0.004*"войско"'),
 (3,
  '0.015*"день" + 0.015*"война" + 0.012*"сегодня" + 0.009*"хлеб" + '
  '0.008*"немец" + 0.007*"час" + 0.007*"говорить" + 0.006*"москва" + '
  '0.006*"город" + 0.006*"получать"'),
 (4,
  '0.012*"час" + 0.010*"улица" + 0.009*"день" + 0.009*"победа" + 0.008*"война" '
  '+ 0.008*"радио" + 0.008*"капитуляция" + 0.007*"площадь" + 0.006*"вечер" + '
  '0.006*"май"'),
 (5,
  '0.009*"день" + 0.009*"война" + 0.007*"мочь" + 0.006*"год" + 0.006*"жизнь" + '


In [46]:

pprint(models[2].print_topics())

[(0,
  '0.007*"год" + 0.005*"сказать" + 0.004*"говорить" + 0.004*"день" + '
  '0.004*"человек" + 0.004*"мочь" + 0.004*"время" + 0.004*"становиться" + '
  '0.003*"идти" + 0.003*"знать"'),
 (1,
  '0.009*"год" + 0.007*"человек" + 0.006*"война" + 0.005*"день" + 0.005*"мочь" '
  '+ 0.005*"жизнь" + 0.004*"жить" + 0.003*"сказать" + 0.003*"время" + '
  '0.003*"любовь"'),
 (2,
  '0.010*"мао" + 0.009*"кпк" + 0.008*"цзэ" + 0.007*"дун" + 0.006*"партия" + '
  '0.005*"съезд" + 0.005*"война" + 0.004*"советский" + 0.004*"цк" + '
  '0.004*"китай"'),
 (3,
  '0.009*"война" + 0.005*"человек" + 0.005*"время" + 0.004*"говорить" + '
  '0.004*"день" + 0.004*"мочь" + 0.004*"год" + 0.004*"армия" + 0.004*"генерал" '
  '+ 0.003*"япония"'),
 (4,
  '0.007*"война" + 0.004*"день" + 0.004*"становиться" + 0.004*"время" + '
  '0.003*"говорить" + 0.003*"человек" + 0.003*"сказать" + 0.003*"первый" + '
  '0.003*"год" + 0.002*"мочь"'),
 (5,
  '0.009*"человек" + 0.005*"год" + 0.005*"советский" + 0.004*"война" + '
  '0.004*"р